## Marker Selection

In [1]:
import pathlib
from concurrent.futures import ProcessPoolExecutor, as_completed
from itertools import combinations
import numpy as np
import anndata
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns
import xarray as xr
import subprocess
from sklearn.metrics import roc_auc_score
from cemba_data.tools.hdf5.anndata import rank_features_groups
import warnings
warnings.filterwarnings('ignore')

## Parameter

In [2]:
min_cluster_cell_number = 10
adj_p_cutoff = 1e-3
top_n = 30000
max_test_cell_population = 1000
delta_rate_cutoff = 0.5
auroc_cutoff = 0.8

In [3]:
# Parameters
cluster_col = "ITSpatialGroup"
tidy_data_path = '/home/hanliu/project/mouse_rostral_brain/study/ITSpatial-DissectionRegion/ITSpatial.cell_tidy_data.hdf'


### Stable Parameter

In [4]:
random_seed = 0

## Cell Tidy Data

In [5]:
cell_tidy_data = pd.read_hdf(tidy_data_path)
cell_tidy_data.shape[0]

24359

In [6]:
records = []
for cluster, sub_df in cell_tidy_data.groupby(cluster_col):
    if sub_df.shape[0] < max_test_cell_population:
        records.append(sub_df)
    else:
        records.append(
            sub_df.sample(max_test_cell_population, random_state=random_seed))
cell_tidy_data = pd.concat(records)
cell_tidy_data[cluster_col].value_counts()

IT-L23+ORB      614
IT-L23+PFC-1    562
IT-L23+ACA-1    539
IT-L23+ACA-3    507
IT-L4+SSp-5     474
               ... 
IT-L5+SSp-1     157
IT-L4+MOs-1     146
IT-L5+SSs-1     143
IT-L5+SSp-2     133
IT-L4+MOs-3     107
Name: ITSpatialGroup, Length: 73, dtype: int64

In [7]:
cluster_series = cell_tidy_data[cluster_col].astype(str)
cluster_counts = cluster_series.value_counts()

def check_cluster(cluster, count):
    if count < min_cluster_cell_number:
        return False
    return True


unique_clusters = [
    cluster for cluster, count in cluster_counts.items()
    if check_cluster(cluster, count)
]
cluster_pairs = list(combinations(unique_clusters, 2))

print(len(unique_clusters), 'pass filter.')
print(len(cluster_pairs), 'pairwise comparison to test.')

73 pass filter.
2628 pairwise comparison to test.


## Gene meta

In [8]:
gene_meta = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusterMethylMarker/gencode.vM22.annotation.gene.flat.filtered_white_genes.tsv.gz',
    index_col='gene_id',
    sep='\t')
gene_meta.index.name = 'gene'
gene_name_to_id = {v: k for k, v in gene_meta['gene_name'].iteritems()}
gene_idbase_to_id = {i.split('.')[0]: i for i in gene_meta.index}

## Pairwise test

In [9]:
meta_path = '/home/hanliu/project/mouse_rostral_brain/study/ITSpatial-DissectionRegion/ITSpatial.cell_tidy_data.hdf'

In [11]:
def get_sig_features(rank_gene_dict):
    pvals_adj = pd.DataFrame(rank_gene_dict['pvals_adj'])
    names = pd.DataFrame(rank_gene_dict['names'])
    return pvals_adj, names


def get_delta(cluster, gene):
    row = cluster_mean_gene_df.loc[gene].copy()
    cluster_value = row.pop(cluster)
    other_mean = row[0] # only two cluster
    delta = cluster_value - other_mean
    return delta


def calculate_single_pair(data_path, pair):
    cluster_a, cluster_b = pair
    output_dir = pathlib.Path(f'TEMP/{cluster_a}')
    output_dir.mkdir(exist_ok=True)
    output_path = output_dir / f'{cluster_b}.msg'
    if output_path.exists():
        return
    
    this_tidy_data = pd.read_hdf(meta_path)
    this_tidy_data = this_tidy_data[this_tidy_data[cluster_col].isin(pair)]
    
    mcds = xr.open_dataarray(data_path).sel(cell=this_tidy_data.index).load()
    adata = anndata.AnnData(X=mcds.values,
                    obs=pd.DataFrame([], mcds.get_index('cell')),
                    var=pd.DataFrame([], mcds.get_index('gene')))
    adata.obs['cluster'] = this_tidy_data[cluster_col].astype('category')
    
    # reverse_adata, centered by 1 because after normalization all prior is center to 1
    adata.X = (adata.X - 1) * -1 + 1
    
    # calculate cluster delta
    records = {}
    for cluster, sub_df in adata.obs.groupby('cluster'):
        sub_adata = adata[sub_df.index, :]
        gene_mean = sub_adata.X.mean(axis=0)
        records[cluster] = pd.Series(gene_mean, index=sub_adata.var_names)
    cluster_mean_gene_df = pd.DataFrame(records)

    # judge gene by delta
    cluster_delta = (cluster_mean_gene_df[cluster_a] -
                     cluster_mean_gene_df[cluster_b]).abs()
    delta_judge = cluster_delta > delta_rate_cutoff
    use_adata = adata[:, delta_judge]
    
    try:
        sc.tl.rank_genes_groups(use_adata,
                            groupby='cluster',
                            n_genes=top_n,
                            method='wilcoxon')
    except:
        print(pair)
        return
    pvals_adj, names = get_sig_features(
        use_adata.uns['rank_genes_groups'])

    results = []
    for col in use_adata.obs['cluster'].unique():
        if col not in pair:
            continue
        df = pd.DataFrame({
            'pvals_adj': pvals_adj[col].tolist(),
            'gene_id': names[col].tolist()
        })
        df['cluster_from'] = col
        df['cluster_to'] = cluster_a if col == cluster_b else cluster_b
        results.append(df)
    
    # get total results filter by adj_p
    total_results = pd.concat(results)
    total_results['gene_name'] = total_results['gene_id'].map(
        gene_meta['gene_name'])
    total_results['-lgp'] = -np.log10(total_results['pvals_adj'])
    total_results['-lgp'] = total_results['-lgp'].replace(np.inf, 1000)
    total_results = total_results[
        total_results['pvals_adj'] < adj_p_cutoff].copy()
    
    # judge by auroc
    total_results['AUROC'] = total_results[[
        'gene_id', 'cluster_from'
    ]].apply(lambda i: get_auroc(i['gene_id'], i['cluster_from'],
                                 use_adata),
             axis=1)
    total_results = total_results[total_results['AUROC'] > auroc_cutoff]
    
    # add rate
    # total_results['cluster_from_rate'] = total_results.apply(
    #     lambda i: cluster_mean_gene_df.at[i['gene_id'], i['cluster_from']],
    #     axis=1)
    # total_results['cluster_to_rate'] = total_results.apply(
    #     lambda i: cluster_mean_gene_df.at[i['gene_id'], i['cluster_to']],
    #     axis=1)
    
    total_results.to_msgpack(output_path)
    return


def get_auroc(gene_id, cluster, adata):
    yscore = adata.obs_vector(gene_id)
    ylabel = adata.obs['cluster'] == cluster
    score = roc_auc_score(ylabel, yscore)
    score = abs(score - 0.5) + 0.5
    return score

In [12]:
data_path = 'ITSpatial.GeneWithSlop2kb.gene_da_rate.nc'

## Run pairwise marker

In [13]:
cpu = 20
chunk_size = 20

In [14]:
pair_marker_counts = {}
n = 0
for chunk_start in range(0, len(cluster_pairs), chunk_size):
    pairs = cluster_pairs[chunk_start : chunk_start + chunk_size]
    
    with ProcessPoolExecutor(cpu) as executor:
        temp_dir = 'TEMP'
        pathlib.Path(temp_dir).mkdir(exist_ok=True)
        futures = []
        for pair in pairs:
            future = executor.submit(calculate_single_pair, data_path, pair)
            futures.append(future)
    
        for future in as_completed(futures):
            n += 1
            if n % 100 == 0:
                print(n)
            future.result()

100
200
('IT-L23+ACA-3', 'IT-L23+ACA-2')
300
400
('IT-L4+SSp-2', 'IT-L4+SSp-1')
500
600
700
800
900
1000
('IT-L23+MOp-3', 'IT-L23+MOp-2')
1100
1200
('IT-L5+PFC-1', 'IT-L5+PFC-2')
1300
1400
1500
('IT-L5+MOp-3', 'IT-L5+MOp-4')
('IT-L5+MOp-3', 'IT-L5+MOp-2')
1600
('IT-L4+SSs-2', 'IT-L4+SSs-1')
('IT-L23+SSs-2', 'IT-L23+SSs-1')
1700
1800
1900
2000
2100
2200
('IT-L6+MOp-4', 'IT-L6+MOp-3')
2300
('IT-L6+MOp-3', 'IT-L6+MOp-2')
('IT-L6+ACA-3', 'IT-L6+ACA-2')


ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

## Aggregate DEG

In [15]:
temp_dir = pathlib.Path('TEMP')
deg_list = list(temp_dir.glob('**/*msg'))

df_list = []
for path in deg_list:
    df_list.append(pd.read_msgpack(path))

total_markers = pd.concat(df_list)

In [30]:
# assert total_markers.set_index(['gene_id', 'cluster_from', 'cluster_to']).index.duplicated().sum() == 0

In [16]:
total_markers.set_index(['gene_id', 'cluster_from', 'cluster_to']).index.duplicated().sum()

0

## Save results

In [17]:
total_markers.to_msgpack('TotalPairwiseMarker.msg')

In [18]:
marker_counts = total_markers.set_index(['cluster_from', 'cluster_to']).index.value_counts()
marker_counts.index = pd.MultiIndex.from_tuples(marker_counts.index.tolist())
marker_counts = marker_counts.reset_index()
marker_counts.columns = ['ClusterA', 'ClusterB', 'GeneCount']
marker_counts.to_csv('Cluster_pair_marker_counts.csv', index=None)

In [19]:
marker_counts[marker_counts['GeneCount'] < 10]

,ClusterA,ClusterB,GeneCount
4840,IT-L6+AI,IT-L6+SSs-1,9
4841,IT-L4+MOp-4,IT-L4+MOp-2,9
4842,IT-L6+SSs-2,IT-L6+MOp-3,9
4843,IT-L6+SSp-1,IT-L6+SSp-5,9
4844,IT-L6+MOs-2,IT-L6+AI,9
4845,IT-L4+MOp-1,IT-L4+MOp-2,9
4846,IT-L6+MOp-3,IT-L6+SSp-3,9
4847,IT-L6+SSp-5,IT-L6+SSp-1,9
4848,IT-L6+MOp-2,IT-L6+SSp-2,9
4849,IT-L6+AI,IT-L6+MOs-2,9


In [20]:
subprocess.run('rm -rf TEMP*', shell=True)

CompletedProcess(args='rm -rf TEMP*', returncode=0)

In [21]:
with open('TotalGeneID.txt', 'w') as f:
    for g in total_markers['gene_id'].unique():
        f.write(f'{g}\n')

In [22]:
total_markers['gene_id'].unique().size

4977